In [ ]:
!pip install -q smolagents
!pip install -q huggingface_hub
!pip -q install litellm

In [ ]:
# from huggingface_hub import login

# login()

In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel

In [ ]:
# from smolagents import CodeAgent, HfApiModel
# from huggingface_hub import login

# login("<YOUR_HUGGINGFACEHUB_API_TOKEN>")

# model_id = "meta-llama/Llama-3.3-70B-Instruct"

# model = HfApiModel(model_id=model_id)
# agent = CodeAgent(tools=[], model=model, add_base_tools=True)

# agent.run(
#     "Could you give me the 118th number in the Fibonacci sequence?",
# )

In [ ]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('openai')

In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, LiteLLMModel

model = LiteLLMModel(model_id="gpt-4o")

agent = CodeAgent(tools=[], model=model, add_base_tools=True)

agent.run("Could you give me the 118th number in the Fibonacci sequence?")

### Code execution
A Python interpreter executes the code on a set of inputs passed along with your tools. This should be safe because the only functions that can be called are the tools you provided (especially if it’s only tools by Hugging Face) and a set of predefined safe functions like print or functions from the math module, so you’re already limited in what can be executed.

The Python interpreter also doesn’t allow imports by default outside of a safe list, so all the most obvious attacks shouldn’t be an issue. You can authorize additional imports by passing the authorized modules as a list of strings in argument additional_authorized_imports upon initialization of your CodeAgent or CodeAgent:

In [ ]:
# access to aditional python packages

from smolagents import CodeAgent

agent = CodeAgent(tools=[], model=model)
agent.run("Could you get me the title of the page at url 'https://huggingface.co/blog'?")

In [ ]:
agent = CodeAgent(tools=[], model=model, additional_authorized_imports=['requests', 'bs4'])
agent.run("Could you get me the title of the page at url 'https://huggingface.co/blog'?")

### System Prompt

In [ ]:
print(agent.system_prompt_template)

The system prompt includes:

- An introduction that explains how the agent should behave and what tools are.
- A description of all the tools that is defined by a {{tool_descriptions}} token that is dynamically replaced at runtime with the tools defined/chosen by the user.
- The tool description comes from the tool attributes, name, description, inputs and output_type, and a simple jinja2 template that you can refine.
- The expected output format.

### Updating the System Prompt

In [ ]:
from smolagents import ToolCallingAgent, PythonInterpreterTool, TOOL_CALLING_SYSTEM_PROMPT

In [ ]:
print(TOOL_CALLING_SYSTEM_PROMPT)

In [ ]:
modified_prompt = TOOL_CALLING_SYSTEM_PROMPT

agent = ToolCallingAgent(tools=[PythonInterpreterTool()], model=model, system_prompt=modified_prompt)

In [ ]:
print(agent.system_prompt_template)

In [ ]:
agent.run("How many seconds would it take for a leopard at full speed to run through Pont des Arts?")

In [ ]:
print(agent.logs)

In [ ]:
agent.logs

In [ ]:
agent.write_inner_memory_from_logs()

### Tools

A tool is an atomic function to be used by an agent. To be used by an LLM, it also needs a few attributes that constitute its API and will be used to describe to the LLM how to call this tool:

- A name
- A description
- Input types and descriptions
- An output type

#### Default toolbox
Transformers comes with a default toolbox for empowering agents, that you can add to your agent upon initialization with argument add_base_tools = True:

- DuckDuckGo web search*: performs a web search using DuckDuckGo browser.
- Python code interpreter: runs your the LLM generated Python code in a secure environment. This tool will only be added to ToolCallingAgent if you initialize it with add_base_tools=True, since code-based agent can already natively execute Python code
- Transcriber: a speech-to-text pipeline built on Whisper-Turbo that transcribes an audio to text.

#### Create a new tool
You can create your own tool for use cases not covered by the default tools from Hugging Face. For example, let’s create a tool that returns the most downloaded model for a given task from the Hub.

In [ ]:
from huggingface_hub import list_models

task = "text-classification"

most_downloaded_model = next(iter(list_models(filter=task, sort="downloads", direction=-1)))

print(most_downloaded_model.id)

In [ ]:
task = "text-to-video"

most_downloaded_model = next(iter(list_models(filter=task, sort="downloads", direction=-1)))

print(most_downloaded_model.id)

In [ ]:
from transformers import tool

@tool
def model_download_tool(task: str) -> str:
    """
    This is a tool that returns the most downloaded model of a given task on the Hugging Face Hub.
    It returns the name of the checkpoint.

    Args:
        task: The task for which
    """
    most_downloaded_model = next(iter(list_models(filter=task, sort="downloads", direction=-1)))
    return most_downloaded_model.id

### The function needs:

- A clear name. The name usually describes what the tool does. Since the code returns the model with the most downloads for a task, let’s put model_download_tool.
- Type hints on both inputs and output
- A description, that includes an ‘Args:’ part where each argument is described (without a type indication this time, it will be pulled from the type hint). All these will be automatically baked into the agent’s system prompt upon initialization: so strive to make them as clear as possible!

In [ ]:
from smolagents import CodeAgent, HfApiModel

agent = CodeAgent(tools=[model_download_tool], model=model)

agent.run("Can you give me the name of the model that has the most downloads in the 'text-to-video' task on the Hugging Face Hub?")

In [ ]:
from smolagents import (
    load_tool,
    CodeAgent,
    HfApiModel,
    GradioUI
)

# Import tool from Hub
image_generation_tool = load_tool("m-ric/text-to-image", trust_remote_code=True)

# model = HfApiModel(model_id)

# Initialize the agent with the image generation tool
agent = CodeAgent(tools=[image_generation_tool], model=model)

GradioUI(agent).launch()

## Orchestrate a multi-agent system

In [ ]:
import re
import requests
from markdownify import markdownify
from requests.exceptions import RequestException
from smolagents import tool


@tool
def visit_webpage(url: str) -> str:
    """Visits a webpage at the given URL and returns its content as a markdown string.

    Args:
        url: The URL of the webpage to visit.

    Returns:
        The content of the webpage converted to Markdown, or an error message if the request fails.
    """
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes

        # Convert the HTML content to Markdown
        markdown_content = markdownify(response.text).strip()

        # Remove multiple line breaks
        markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)

        return markdown_content

    except RequestException as e:
        return f"Error fetching the webpage: {str(e)}"
    except Exception as e:
        return f"An unexpected error occurred: {str(e)}"

In [ ]:
print(visit_webpage("https://en.wikipedia.org/wiki/Hugging_Face")[:500])

In [ ]:
from smolagents import (
    CodeAgent,
    ToolCallingAgent,
    HfApiModel,
    ManagedAgent,
    DuckDuckGoSearchTool,
    LiteLLMModel
)

# model = HfApiModel(model_id)

web_agent = ToolCallingAgent(
    tools=[DuckDuckGoSearchTool(), visit_webpage],
    model=model,
    # max_iterations=10
)

In [ ]:
managed_web_agent = ManagedAgent(
    agent=web_agent,
    name="search",
    description="Runs web searches for you. Give it your query as an argument."
)

In [ ]:
manager_agent = CodeAgent(
    tools=[],
    model=model,
    managed_agents=[managed_web_agent],
    additional_authorized_imports=["time", "numpy", "pandas"]
)

In [ ]:
answer = manager_agent.run("Which dynasty was ruling China at the time of the fall of Constantinople?.")

In [ ]:
answer = manager_agent.run("Which dynasty was ruling China at the time of the fall of Constantinople?.")

In [ ]:
answer = manager_agent.run("If LLM trainings continue to scale up at the current rythm until 2030, what would be the electric power in GW required to power the biggest training runs by 2030? What does that correspond to, compared to some contries? Please provide a source for any number used.")